In [10]:
import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass

In [11]:
# -------------------------------------------
# Modify the holiday class to 
# 1. Only accept Datetime objects for date.
# 2. You may need to add additional functions
# 3. You may drop the init if you are using @dataclasses
# --------------------------------------------

    
class Holiday:
    # def __init__(self, name : str, date : datetime.date):

    def __init__(self, name, date):
        #Your Code Here   
        self._name = name 
        self._date = date 
    

    def __str__ (self):
        # String output
        # Holiday output when printed.
        return f"{self._name} ({self._date.strftime('%Y-%m-%d')})"

    @property
    def name(self):
        return self._name
    
    @property
    def date(self):
        return self._date
          

In [12]:
# -------------------------------------------
# The HolidayList class acts as a wrapper and container
# For the list of holidays
# Each method has pseudo-code instructions
# --------------------------------------------


class HolidayList:
    def __init__(self):
       self.innerHolidays = []
   
    def addHoliday(self, holidayObj):
        # Make sure holidayObj is an Holiday Object by checking the type
        if (isinstance(holidayObj,Holiday)):
        # Use innerHolidays.append(holidayObj) to add holiday
            self.innerHolidays.append(holidayObj)
        # print to the user that you added a holiday
            print(f'Success: {holidayObj} has been added to the holiday list.')

    def findHoliday(self, HolidayName, Date):
        # Find Holiday in innerHolidays
        for holiday in self.innerHolidays:
            if (holiday.name == HolidayName and holiday.date == Date):
                return holiday
        # Return Holiday

    def removeHoliday(self, HolidayName, Date):
        # Find Holiday in innerHolidays by searching the name and date combination.
        holiday = self.findHoliday(HolidayName, Date)
        # remove the Holiday from innerHolidays
        self.innerHolidays.remove(holiday)
        # inform user you deleted the holiday
        print(f'Sucess: \n{holiday} has been removed from the holiday list.')

    def read_json(self, filelocation):
        # Read in things from json file location
        with open(filelocation, 'r') as f:
            holidays = json.load(f)
        # Use addHoliday function to add holidays to inner list.
        for holiday_dict in holidays["holidays"]:
            name = holiday_dict["name"]
            date = datetime.datetime.strptime(holiday_dict["date"],"%Y-%m-%d")
            self.addHoliday(Holiday(name,date))

    def save_to_json(self, filelocation):
        # Write out json file to selected file.
        holiday_list = []
        for i in self.innerHolidays:
            holiday = {'name': i.name, 'date': i.date.strftime('%Y-%m-%d')}
            holiday_list.append(holiday)
        saved_dict = {'holidays': holiday_list}
        with open(filelocation, 'w') as f:
            json.dump(saved_dict, f, indent = 4)


    def scrapeHolidays(self):
        # Scrape Holidays from https://www.timeanddate.com/holidays/us/ 
        # Remember, 2 previous years, current year, and 2  years into the future. You can scrape multiple years by adding year to the timeanddate URL. For example https://www.timeanddate.com/holidays/us/2022
        # Check to see if name and date of holiday is in innerHolidays array
        # Add non-duplicates to innerHolidays
        # Handle any exceptions.     
        year = 2022
        url = 'https://www.timeanddate.com/holidays/us/'
        holidays = set()

        for year in range(year - 2, year + 3):
            year_url = url + str(year)
            request = requests.get(year_url).text
            html = BeautifulSoup(request, 'html.parser')

            table = html.find('table').find('tbody')
            rows = table.find_all('tr')

            for row in rows:
                if len(row.contents) == 0:
                    continue

                date_str = row.find('th').contents[0] + ' ' + str(year)
                format = '%b %d %Y'
                # date = str(datetime.datetime.strptime(date_str, format).date())
                date = datetime.datetime.strptime(date_str, format)
                
                name = row.find_all('td')[1].find('a').contents[0]

                # holidays.add((name, date))
                self.addHoliday(Holiday(name,date))                
        


    def numHolidays(self):
        # Return the total number of holidays in innerHolidays
        return len(self.innerHolidays)

    def filter_holidays_by_week(self, year, week_number):
        # Use a Lambda function to filter by week number and save this as holidays, use the filter on innerHolidays
        # Week number is part of the the Datetime object
        # Cast filter results as list
        # return your holidays
        holidayList = list(filter(lambda x: x.date.isocalendar().week == week_number and x.date.isocalendar().year == year, self.innerHolidays))
        return holidayList


    def displayHolidaysInWeek(self, holidayList):
        # Use your filter_holidays_by_week to get list of holidays within a week as a parameter
        # Output formated holidays in the week. 
        # * Remember to use the holiday __str__ method.    
        for holiday in holidayList:
                print (holiday)


    def viewCurrentWeek(self):
        # Use the Datetime Module to look up current week and year
        current_year = datetime.datetime.now().year
        current_week = datetime.datetime.now().isocalendar().week
        # Use your filter_holidays_by_week function to get the list of holidays 
        # for the current week/year
        holidayList = self.filter_holidays_by_week(current_year, current_week)
        # Use your displayHolidaysInWeek function to display the holidays in the week
        self.displayHolidaysInWeek(holidayList)
        
        



In [13]:
def main():
    # Large Pseudo Code steps
    # -------------------------------------
    # 1. Initialize HolidayList Object
    init_holidays = HolidayList()
    # 2. Load JSON file via HolidayList read_json function
    init_holidays.read_json('holidays.json')
    # 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
    init_holidays.scrapeHolidays()
    # 3. Create while loop for user to keep adding or working with the Calender
    # 4. Display User Menu (Print the menu)
    print('Welcome to the Holiday Manager!')
    print(f'There are {init_holidays.numHolidays()} holidays stored in the system.')
    
    tasks = True
    while tasks == True: 
        print("Holiday Manager \n===============")
        print("1. Add a Holiday \n2. Remove a Holiday \n3. Save Holiday List \n4. View Holidays \n5. Exit")
    # 5. Take user input for their action based on Menu and check the user input for errors
        choose_task = int(input("Please enter the corresponding number for the task you want to choose:"))
    # 6. Run appropriate method from the HolidayList object depending on what the user input is
        if choose_task == 1:
            print("Add a Holiday \n=============")
            name = input("Please enter the holiday's name:")
            date_str = input("Please enter the holiday's date (YYYY-MM-DD):")
            date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
            init_holidays.addHoliday(Holiday(name, date))
        
        elif choose_task == 2:
            print("Remove a Holiday \n================")
            name = input("Please enter the holiday's name:")
            date_str = input("Please enter the holiday's date (YYYY-MM-DD):")
            date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
            init_holidays.removeHoliday(name, date)

        elif choose_task == 3:
            print("Saving Holiday List \n==================")
            save_input = input("Are you sure you want to save your changes? [y/n]")
            if save_input.lower() == "y":
                init_holidays.save_to_json('HolidayList.json')  
                print("Success: \nYour changes have been saved.")
            else:
                print("Cancelled: \nHoliday list file save was cancelled.")

        elif choose_task == 4:
            print("View Holidays \n=============")
            year = int(input("Which year (2020-2024)? If you want to see the current week, please enter 0:"))
            week_number = int(input("Which week (1-52)? If you want to see the current week, please enter 0:"))
            if week_number == 0 or year == 0:
                init_holidays.viewCurrentWeek()
            else:
                init_holidays.displayHolidaysInWeek(init_holidays.filter_holidays_by_week(year,week_number))

        elif choose_task == 5: 
            exit_input = (input("Are you sure you want to exit? [y/n]").lower())
            if exit_input == "y":
                tasks = False 
                break
            else:
                tasks = True 

        else:
            print('Not a valid input. Please enter a number from 1 to 5:')

    # 7. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going. 

In [14]:
main()

Success: Margaret Thatcher Day (2021-01-10) has been added to the holiday list.
Success: World Sketchnote Day (2021-01-11) has been added to the holiday list.
Success: Zanzibar Revolution Day (2021-01-12) has been added to the holiday list.
Success: National Rubber Ducky Day (2021-01-13) has been added to the holiday list.
Success: Tamil Thai Pongdal Day (2021-01-14) has been added to the holiday list.
Success: National Bagel Day (2021-01-15) has been added to the holiday list.
Success: Signing of the Peace Accords (2021-01-16) has been added to the holiday list.
Success: New Year's Day (2020-01-01) has been added to the holiday list.
Success: World Braille Day (2020-01-04) has been added to the holiday list.
Success: Epiphany (2020-01-06) has been added to the holiday list.
Success: Asarah B'Tevet (2020-01-07) has been added to the holiday list.
Success: Orthodox Christmas Day (2020-01-07) has been added to the holiday list.
Success: International Programmers' Day (2020-01-07) has bee

In [6]:
# Additional Hints:
# ---------------------------------------------
# You may need additional helper functions both in and out of the classes, add functions as you need to.
#
# No one function should be more then 50 lines of code, if you need more then 50 lines of code
# excluding comments, break the function into multiple functions.
#
# You can store your raw menu text, and other blocks of texts as raw text files 
# and use placeholder values with the format option.
# Example:
# In the file test.txt is "My name is {fname}, I'm {age}"
# Then you later can read the file into a string "filetxt"
# and substitute the placeholders 
# for example: filetxt.format(fname = "John", age = 36)
# This will make your code far more readable, by seperating text from code.